In [ ]:
import json
import numpy as np
from openai import OpenAI
from typing import List
import faiss
import pickle
from dotenv import load_dotenv

In [ ]:
# 환경 변수 설정 (.env 또는 직접 설정)
load_dotenv()

client = OpenAI()

filename = ["skinner", "bowlby", "piaget"]
# 2. 텍스트 파싱
raw_text = []
for n in filename:
    with open(f"texts/{n}.json", "r", encoding="utf-8") as f:
        raw_text += json.load(f)
        print(len(raw_text))

In [ ]:
docs = [text["content"] for text in raw_text]
metadatas = [{k: v for k, v in text.items() if k != "content"} for text in raw_text]

author = []
for data in metadatas:
    if data["author"] not in author:
        author.append(data["author"])

print(author)

In [ ]:
# OpenAI Embedding
def get_embeddings(texts: List[str]) -> List[List[float]]:
    response = client.embeddings.create(input=texts, model="text-embedding-3-small")
    return [e.embedding for e in response.data]


embeddings = get_embeddings(docs)

# FAISS 벡터스토어 저장
dimension = len(embeddings[0])
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings).astype("float32"))

# 텍스트 + 메타데이터 함께 저장
chunks = [{"doc": doc, **meta} for doc, meta in zip(docs, metadatas)]

with open("faiss_chunks.pkl", "wb") as f:
    pickle.dump(chunks, f)

faiss.write_index(index, "faiss_index.idx")
print("✅ 벡터스토어 저장 완료")

In [ ]:
# 5. 검색 함수
def search(query: str, k: int = 3):
    q_embedding = get_embeddings([query])[0]
    D, I = index.search(np.array([q_embedding]).astype("float32"), k)
    with open("faiss_chunks.pkl", "rb") as f:
        stored_chunks = pickle.load(f)
    return [stored_chunks[i] for i in I[0]]

In [64]:
results = search("선생님이랑 사이가 안 좋은 아이", k=3)
for r in results:
    print(r["author"], r["title"])
    print(r["doc"])

John Bowlby 민감한 양육 (Maternal Sensitivity / Responsive Caregiving)
볼비와 아인스워스는 양육자의 **민감성(Maternal Sensitivity)**이 안정 애착 형성의 핵심 요인이라고 강조했습니다. 민감한 양육이란 양육자가 유아의 신호(울음, 미소, 몸짓 등)에 **주의를 기울이고, 정확하게 해석하며, 적절하고 즉각적으로 반응**하는 능력을 의미합니다. 예를 들어, 아이가 배고파서 울 때 즉시 수유하고, 놀고 싶어 미소 지을 때 함께 놀아주는 것이 민감한 양육의 예입니다. 양육자의 이러한 민감한 반응은 아이에게 '내가 도움을 요청하면, 부모는 나를 이해하고 도와줄 것이다'라는 신뢰감을 형성하게 하여 긍정적인 내적 작동 모델과 안정적인 애착을 발달시킵니다. 반대로, 비민감하거나 예측 불가능한 양육은 불안정 애착 형성으로 이어질 수 있습니다. 민감성은 타고나는 것이 아니라 학습될 수 있는 기술이며, 부모 교육 프로그램을 통해 향상될 수 있습니다.
Jean Piaget 조절 (Accommodation)
조절은 아동이 새로운 정보를 이해하기 위해 기존 스키마를 수정하거나 새로운 스키마를 창출하는 인지적 과정이다. 이는 기존 스키마로는 더 이상 설명이 불가능한 경험이 주어졌을 때, 사고 구조를 재구성하는 방식으로 작동한다. 조절은 기존의 틀을 유지하려는 동화와 달리, 자신의 사고방식을 유연하게 바꾸는 능동적인 학습 과정이다. 예를 들어, 아동이 고양이를 처음 보고 강아지라고 했다가 '이건 고양이야'라는 피드백을 통해 '고양이'라는 새로운 스키마를 형성하게 되는 것이 조절이다. 조절은 기존의 이해 구조에 충격을 주는 정보를 마주했을 때 일어나며, 이는 인지적 균형 상태를 다시 찾기 위한 자연스러운 반응이다. 피아제는 조절이 없이는 진정한 학습과 발달이 일어날 수 없다고 보았다. 조절은 아동의 인지 구조가 더 정교해지고 현실 세계에 보다 정확하게 적응하도록 돕는다.
B.F. Skinner 복잡한 행동 가르치기: 행동 조성